In [1]:
%run prepareStats.py

Last GP in the database: 2022 Hungarian Grand Prix


In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
results = results[~((results["year"] < 1961) & (results["location"] == "Indianapolis"))]

In [4]:
results = results[results["laps"] > 0]

In [5]:
allDrivers = results["name"].drop_duplicates().to_list()

In [6]:
allDriversOnPodium = podiums["name"].drop_duplicates().to_list()

In [7]:
results["raced"] = results["raceId"].map(results.groupby("raceId")["name"].agg(list))

In [8]:
podiums["sharedPodium"] = podiums["raceId"].map(podiums.groupby("raceId")["name"].agg(list))

C:\Users\micha\AppData\Local\Temp\ipykernel_15120\3814610795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  podiums["sharedPodium"] = podiums["raceId"].map(podiums.groupby("raceId")["name"].agg(list))


In [9]:
decidingraces = pd.read_csv(os.path.join("dataHandcollected", "titles.csv"))
decidingraces["date"] = pd.to_datetime(decidingraces["date"], format="%Y-%m-%d")
decidingraces["decidingRace"] = True
results = results.merge(decidingraces, on = ["date"], how = "left")

In [10]:
champions = results["champion"].drop_duplicates().to_list()

In [11]:
dateOfDebut = pd.Series(results.groupby(["driverId"])["date"].min(), name="dateOfDebut")
dateOfFarewell = pd.Series(results.groupby(["driverId"])["date"].max(), name="dateOfFarewell")
results = results.merge(dateOfDebut, on = ["driverId"], how = "left")
results = results.merge(dateOfFarewell, on = ["driverId"], how = "left")

## How many other World Champions has each World Champion competed against?

In [12]:
championAdversaries = {}
for champion in champions:
    championsRacedwith = []
    racedwith = results[results["name"] == champion]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in champions:
            championsRacedwith.append(x)
    for x in championsRacedwith:
        if x == champion:
            championsRacedwith.remove(x)
    racedNames = championsRacedwith
    racedCount = len(championsRacedwith)
    championAdversaries[champion] = [racedCount, ", ".join(racedNames)]
championAdversaries = pd.DataFrame.from_dict(championAdversaries, orient="index", columns=["count", "champions"])

championAdversaries.sort_values(by="count",ascending=False).head(10)

## How many drivers has each driver shared podiums with?

Technically speaking, some races had different drivers on podium than in the top-3 of the official results: DSQs etc. But let's find out anyway.

In [13]:
sharedPodiums = {}
sharedPodiumsCount = {}
for driver in allDriversOnPodium:
    sharedwith = podiums[podiums["name"] == driver]
    sharedwith = sharedwith["sharedPodium"].drop_duplicates().tolist()
    sharedwith = [item for sublist in sharedwith for item in sublist]
    sharedwith = list(set(sharedwith))
    for x in sharedwith:
        if x == driver:
            sharedwith.remove(x)
    sharedPodiums[driver] = sharedwith
    sharedPodiumsCount[driver] = [len(sharedwith), ", ".join(sharedwith)]
sharedPodiumsCount = pd.DataFrame.from_dict(sharedPodiumsCount, orient="index", columns=["count", "names"])    

In [14]:
sharedPodiumsCount.sort_values(by="count",ascending=False).head(10)

,count,names
Alain Prost,31,"John Watson, Niki Lauda, Ivan Capelli, Didier Pironi, Derek Warwick, Mark Blundell, Damon Hill, Nigel Mansell, Gerhard Berger, Riccardo Patrese, Mika Häkkinen, Patrick Tambay, Michele Alboreto, Jacques Laffite, Nelson Piquet, Michael Schumacher, René Arnoux, Ayrton Senna, Martin Brundle, Bruno Giacomelli, Elio de Angelis, Alessandro Nannini, Alan Jones, Maurício Gugelmin, Stefan Johansson, Andrea de Cesaris, Thierry Boutsen, Keke Rosberg, Carlos Reutemann, Eddie Cheever, Stefano Modena"
Michael Schumacher,31,"Nick Heidfeld, Jenson Button, Mark Blundell, Robert Kubica, Damon Hill, Nigel Mansell, Gerhard Berger, Riccardo Patrese, Jarno Trulli, Mika Häkkinen, Jacques Villeneuve, Giancarlo Fisichella, Jos Verstappen, Martin Brundle, Felipe Massa, Heinz-Harald Frentzen, Nicola Larini, Ayrton Senna, Ralf Schumacher, Alexander Wurz, Eddie Irvine, Johnny Herbert, Takuma Sato, Rubens Barrichello, Kimi Räikkönen, Fernando Alonso, David Coulthard, Alain Prost, Jean Alesi, Juan Pablo Montoya, Tiago Monteiro"
Niki Lauda,28,"John Watson, Didier Pironi, Derek Warwick, Nigel Mansell, Riccardo Patrese, James Hunt, Mario Andretti, Gunnar Nilsson, Patrick Tambay, Michele Alboreto, Jody Scheckter, Jacques Laffite, Nelson Piquet, René Arnoux, Ayrton Senna, Denny Hulme, Ronnie Peterson, Emerson Fittipaldi, Jochen Mass, Hans-Joachim Stuck, Alan Jones, Tom Pryce, Patrick Depailler, Carlos Pace, Clay Regazzoni, Alain Prost, Keke Rosberg, Carlos Reutemann"
Lewis Hamilton,27,"Lando Norris, Charles Leclerc, Nico Rosberg, Nick Heidfeld, Jenson Button, Robert Kubica, Sebastian Vettel, Valtteri Bottas, Mark Webber, Jarno Trulli, Romain Grosjean, Daniel Ricciardo, Nelson Piquet Jr., Sergio Pérez, George Russell, Alexander Wurz, Max Verstappen, Alexander Albon, Esteban Ocon, Carlos Sainz, Rubens Barrichello, Heikki Kovalainen, Vitaly Petrov, Kimi Räikkönen, Timo Glock, Fernando Alonso, Felipe Massa"
Juan Fangio,27,"Hans Herrmann, Jean Behra, Stirling Moss, Luigi Musso, Piero Taruffi, Karl Kling, Peter Collins, Mike Hawthorn, Masten Gregory, Luigi Fagioli, Ron Flockhart, Maurice Trintignant, Peter Whitehead, Carlos Menditeguy, Luigi Villoresi, Eugenio Castellotti, Nino Farina, Alberto Ascari, Louis Rosier, Tony Brooks, Wolfgang von Trips, José Froilán González, Louis Chiron, Harry Schell, Robert Manzon, Alfonso de Portago, Umberto Maglioli"
Carlos Reutemann,26,"John Watson, Niki Lauda, Didier Pironi, Jean-Pierre Beltoise, Jean-Pierre Jabouille, Nigel Mansell, Riccardo Patrese, James Hunt, Mario Andretti, Mike Hailwood, François Cevert, Gilles Villeneuve, Jody Scheckter, Jacques Laffite, Nelson Piquet, Jacky Ickx, René Arnoux, Denny Hulme, Ronnie Peterson, Emerson Fittipaldi, Jochen Mass, Alan Jones, Patrick Depailler, Carlos Pace, Clay Regazzoni, Alain Prost"
Kimi Räikkönen,25,"Nick Heidfeld, Nico Rosberg, Jenson Button, Robert Kubica, Sebastian Vettel, Valtteri Bottas, Mark Webber, Jarno Trulli, Lewis Hamilton, Romain Grosjean, Daniel Ricciardo, Giancarlo Fisichella, Michael Schumacher, Felipe Massa, Heinz-Harald Frentzen, Sergio Pérez, Ralf Schumacher, Max Verstappen, Rubens Barrichello, Heikki Kovalainen, Timo Glock, Fernando Alonso, Pastor Maldonado, David Coulthard, Juan Pablo Montoya"
Nelson Piquet,25,"John Watson, Niki Lauda, Didier Pironi, Nigel Mansell, Gerhard Berger, Riccardo Patrese, Patrick Tambay, Michele Alboreto, Jacques Laffite, Aguri Suzuki, René Arnoux, Ayrton Senna, Emerson Fittipaldi, Elio de Angelis, Alan Jones, Stefan Johansson, Andrea de Cesaris, Roberto Moreno, Thierry Boutsen, Alain Prost, Keke Rosberg, Carlos Reutemann, Teo Fabi, Eddie Cheever, Stefano Modena"
Ayrton Senna,24,"Niki Lauda, Ivan Capelli, Derek Warwick, Mark Blundell, Damon Hill, Nigel Mansell, Gerhard Berger, Riccardo Patrese, Mika Häkkinen, Patrick Tambay, Michele Alboreto, Martin Brundle, Michael Schumacher, Nelson Piquet, Jacques Laffite, René Arnoux, Alessandro Nannini, Jyrki Järvilehto, Stefan Johansson, Thierry Boutsen, Alain Prost, K

## What's the longest span between the first race with a driver who would compete with driver X and the last race who has competed against the same driver? AKA the F1 Wormhole.

In [15]:
adversaries = {}
for driver in allDrivers:
    earliest = results["dateOfDebut"].max()
    latest = results["dateOfFarewell"].min()
    racedwith = results[results["name"] == driver]
    dateOfDebut = racedwith["dateOfDebut"].min()
    dateOfFarewell = racedwith["dateOfFarewell"].max()
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x == driver:
            racedwith.remove(x)
        else:
            earliestDebut = results[results["name"] == x]["dateOfDebut"].min()
            latestFarewell = results[results["name"] == x]["dateOfFarewell"].max()
            if earliestDebut < earliest:
                earliest = earliestDebut
            if latestFarewell > latest:
                latest = latestFarewell
    racedNames = racedwith
    racedCount = len(racedwith)
    adversaries[driver] = [dateOfDebut, dateOfFarewell, racedCount, ", ".join(racedNames), earliest, latest]
adversaries = pd.DataFrame.from_dict(adversaries, orient="index", columns=["first GP", "last GP", "count", "names", "earliest", "latest"])
adversaries.index.name = "name"
adversaries["span"] = adversaries["latest"] - adversaries["earliest"]

In [16]:
adversaries[["first GP","last GP","count","earliest","latest","span"]].sort_values(by="span",ascending=False).head(10)

,first GP,last GP,count,earliest,latest,span
name,,,,,,
Nelson Piquet,1978-07-30,1991-11-03,119,1967-08-06,2012-11-25,16548 days
Jan Lammers,1979-01-21,1992-11-08,70,1967-08-06,2012-11-25,16548 days
Riccardo Patrese,1977-05-22,1993-11-07,147,1967-08-06,2012-11-25,16548 days
Michael Schumacher,1991-09-08,2012-11-25,143,1977-05-22,2022-07-31,16506 days
Rubens Barrichello,1993-03-14,2011-11-27,136,1977-05-22,2022-07-31,16506 days
Jean Alesi,1989-07-09,2001-10-14,108,1977-05-22,2022-07-31,16506 days
Mika Häkkinen,1991-03-10,2001-10-14,97,1977-05-22,2022-07-31,16506 days
Luca Badoer,1993-03-14,2009-08-30,77,1977-05-22,2022-07-31,16506 days
Eddie Irvine,1993-10-24,2002-10-13,86,1977-05-22,2022-07-31,16506 days


In [17]:
adversaries[["count"]].sort_values(by="count",ascending=False).head(10)

,count
name,
Graham Hill,236
Maurice Trintignant,235
Jack Brabham,214
Jo Bonnier,206
Stirling Moss,195
Roy Salvadori,187
Harry Schell,176
Bruce McLaren,173
John Surtees,168


In [18]:
lastGPdriversAdversaries = adversaries[adversaries.index.isin(lastGPdrivers)]

In [19]:
lastGPdriversAdversaries[["count"]].sort_values(by="count",ascending=False).head(10)

,count
name,
Fernando Alonso,112
Lewis Hamilton,84
Sebastian Vettel,84
Sergio Pérez,69
Daniel Ricciardo,69
Valtteri Bottas,55
Kevin Magnussen,48
Carlos Sainz,45
Max Verstappen,44


## Fangio Number

Inspired by the [Erdős number](https://en.wikipedia.org/wiki/Erd%C5%91s_number): J-M Fangio's own Fangio number was 0. Each driver who has raced against JMF at least for one lap has Fangio number 1, each driver who has raced against anyone with Fangio number 1 has Fangio number 2 etc.

In [20]:
fangio0 = ["Juan Fangio"]

In [21]:
fangio1, fangio2, fangio3, fangio4, fangio5, fangio6, fangio7, alreadycounted = [], [], [], [], [], [], [], []
for driver in allDrivers:
    racedwith = results[results["name"] == driver]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in fangio0:
            if driver not in fangio0:
                fangio1.append(driver)
                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio1:
                if driver not in fangio0:
                    if driver not in fangio1:
                        fangio2.append(driver)
                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio2:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            fangio3.append(driver)
                            alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio3:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                fangio4.append(driver)
                                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio4:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    fangio5.append(driver)
                                    alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio5:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    if driver not in fangio5:
                                        fangio6.append(driver)
                                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in fangio6:
                if driver not in fangio0:
                    if driver not in fangio1:
                        if driver not in fangio2:
                            if driver not in fangio3:
                                if driver not in fangio4:
                                    if driver not in fangio5:
                                        if driver not in fangio6:
                                            fangio7.append(driver)
                                            alreadycounted.append(driver)

In [22]:
results.loc[results.name.isin(fangio0),"fangioNumber"]=0
results.loc[results.name.isin(fangio1),"fangioNumber"]=1
results.loc[results.name.isin(fangio2),"fangioNumber"]=2
results.loc[results.name.isin(fangio3),"fangioNumber"]=3
results.loc[results.name.isin(fangio4),"fangioNumber"]=4
results.loc[results.name.isin(fangio5),"fangioNumber"]=5
results.loc[results.name.isin(fangio6),"fangioNumber"]=6

In [23]:
fangioNumbers = {}
for x in range(0,7):
    fangioResults = results[results["fangioNumber"] == x]
    fangioResults.sort_values(by="date")
    fangioGeneration = fangioResults["name"].drop_duplicates().to_list()
    earliest = fangioResults["date"].min()
    earliestDrivers = fangioResults[fangioResults["date"] == earliest]
    earliestDrivers = earliestDrivers["name"].to_list()
    latest = fangioResults["date"].max()
    latestDrivers = fangioResults[fangioResults["date"] == latest]
    latestDrivers = latestDrivers["name"].to_list()
    drivers = len(fangioGeneration)
    fangioChamps = []
    for champ in champions:
        if champ in fangioGeneration:
            fangioChamps.append(champ)
    fanChamps = len(fangioChamps)
    fangioNumbers[x] = [drivers, earliest, ", ".join(earliestDrivers), latest, ", ".join(latestDrivers), fanChamps, ", ".join(fangioChamps)]
fangioNumbers = pd.DataFrame.from_dict(fangioNumbers, orient="index", columns=["drivers", "earliest", "earliest drivers", "latest", "latest drivers", "no_champs", "champions"])

In [24]:
fangioNumbers[["earliest","earliest drivers","latest","latest drivers"]]

,earliest,earliest drivers,latest,latest drivers
0,1950-05-13,Juan Fangio,1958-07-06,Juan Fangio
1,1950-05-13,"Louis Chiron, Bob Gerard, Louis Rosier, Toulo de Graffenried, Nino Farina, Johnny Claes, Peter Walker, Tony Rolt, Prince Bira, Reg Parnell, Yves Cabantous, Philippe Étancelin, David Murray, Joe Kelly, Brian Shawe Taylor, David Hampshire, Geoff Crossley, Luigi Fagioli, Cuth Harrison, Joe Fry, Eugène Martin, Leslie Johnson",1975-01-26,Graham Hill
2,1952-05-18,Toni Ulmen,1986-07-06,Jacques Laffite
3,1973-02-11,Luiz Bueno,1997-10-26,Gerhard Berger
4,1986-09-07,Alex Caffi,2012-11-25,Michael Schumacher
5,1997-06-15,Alexander Wurz,2022-07-31,"Lewis Hamilton, Fernando Alonso, Sebastian Vettel, Sergio Pérez, Daniel Ricciardo"
6,2013-03-17,"Max Chilton, Esteban Gutiérrez, Valtteri Bottas, Giedo van der Garde, Jules Bianchi",2022-07-31,"Valtteri Bottas, Kevin Magnussen, Max Verstappen, Carlos Sainz, Esteban Ocon, Lance Stroll, Pierre Gasly, Charles Leclerc, Lando Norris, George Russell, Alexander Albon, Nicholas Latifi, Yuki Tsunoda, Mick Schumacher, Guanyu Zhou"


In [25]:
fangioNumbers[["no_champs", "champions"]]

,no_champs,champions
0,1,Juan Fangio
1,6,"Graham Hill, Mike Hawthorn, Phil Hill, Jack Brabham, Alberto Ascari, Nino Farina"
2,10,"Niki Lauda, Mario Andretti, Jody Scheckter, James Hunt, Emerson Fittipaldi, Jackie Stewart, Jochen Rindt, Denny Hulme, Jim Clark, John Surtees"
3,6,"Alain Prost, Ayrton Senna, Nigel Mansell, Nelson Piquet, Keke Rosberg, Alan Jones"
4,4,"Michael Schumacher, Mika Häkkinen, Jacques Villeneuve, Damon Hill"
5,6,"Lewis Hamilton, Jenson Button, Sebastian Vettel, Kimi Räikkönen, Nico Rosberg, Fernando Alonso"
6,1,Max Verstappen


In [26]:
fangioNumbers["drivers"].sum()

669

## Farina Number

It could also make sense to categorise all the drivers by their Farina Number – Giuseppe Farina won the first GP ever and was the first World Champion.

In [27]:
farina0 = ["Nino Farina"]

In [28]:
farina1, farina2, farina3, farina4, farina5, farina6, farina7, alreadycounted = [], [], [], [], [], [], [], []
for driver in allDrivers:
    racedwith = results[results["name"] == driver]
    racedwith = racedwith["raced"].drop_duplicates().tolist()
    racedwith = [item for sublist in racedwith for item in sublist]
    racedwith = list(set(racedwith))
    for x in racedwith:
        if x in farina0:
            if driver not in farina0:
                farina1.append(driver)
                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina1:
                if driver not in farina0:
                    if driver not in farina1:
                        farina2.append(driver)
                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina2:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            farina3.append(driver)
                            alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina3:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                farina4.append(driver)
                                alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina4:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    farina5.append(driver)
                                    alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina5:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    if driver not in farina5:
                                        farina6.append(driver)
                                        alreadycounted.append(driver)
for driver in allDrivers:
    if driver not in alreadycounted:
        racedwith = results[results["name"] == driver]
        racedwith = racedwith["raced"].drop_duplicates().tolist()
        racedwith = [item for sublist in racedwith for item in sublist]
        racedwith = list(set(racedwith))
        for x in racedwith:
            if x in farina6:
                if driver not in farina0:
                    if driver not in farina1:
                        if driver not in farina2:
                            if driver not in farina3:
                                if driver not in farina4:
                                    if driver not in farina5:
                                        if driver not in farina6:
                                            farina7.append(driver)
                                            alreadycounted.append(driver)

In [29]:
results.loc[results.name.isin(farina0),"farinaNumber"]=0
results.loc[results.name.isin(farina1),"farinaNumber"]=1
results.loc[results.name.isin(farina2),"farinaNumber"]=2
results.loc[results.name.isin(farina3),"farinaNumber"]=3
results.loc[results.name.isin(farina4),"farinaNumber"]=4
results.loc[results.name.isin(farina5),"farinaNumber"]=5
results.loc[results.name.isin(farina6),"farinaNumber"]=6
results.loc[results.name.isin(farina7),"farinaNumber"]=7

In [30]:
farinaNumbers = {}
for x in range(0,8):
    farinaResults = results[results["farinaNumber"] == x]
    farinaResults.sort_values(by="date")
    farinaGeneration = farinaResults["name"].drop_duplicates().to_list()
    earliest = farinaResults["date"].min()
    earliestDrivers = farinaResults[farinaResults["date"] == earliest]
    earliestDrivers = earliestDrivers["name"].to_list()
    latest = farinaResults["date"].max()
    latestDrivers = farinaResults[farinaResults["date"] == latest]
    latestDrivers = latestDrivers["name"].to_list()
    drivers = len(farinaGeneration)
    farinaChamps = []
    for champ in champions:
        if champ in farinaGeneration:
            farinaChamps.append(champ)
    fanChamps = len(farinaChamps)
    farinaNumbers[x] = [drivers, earliest, ", ".join(earliestDrivers), latest, ", ".join(latestDrivers), fanChamps, ", ".join(farinaChamps)]
farinaNumbers = pd.DataFrame.from_dict(farinaNumbers, orient="index", columns=["drivers", "earliest", "earliest drivers", "latest", "latest drivers", "no_champs", "champions"])

In [31]:
farinaNumbers[["drivers", "earliest","earliest drivers","latest","latest drivers"]]

,drivers,earliest,earliest drivers,latest,latest drivers
0,1,1950-05-13,Nino Farina,1955-06-05,Nino Farina
1,137,1950-05-13,"Juan Fangio, Louis Chiron, Bob Gerard, Louis Rosier, Toulo de Graffenried, Johnny Claes, Peter Walker, Tony Rolt, Prince Bira, Reg Parnell, Yves Cabantous, Philippe Étancelin, David Murray, Joe Kelly, Brian Shawe Taylor, David Hampshire, Geoff Crossley, Luigi Fagioli, Cuth Harrison, Joe Fry, Eugène Martin, Leslie Johnson",1964-09-06,Maurice Trintignant
2,132,1954-07-17,"Ron Flockhart, Horace Gould, Leslie Marr, Don Beauman, Leslie Thorne, Bill Whitehouse, John Riseley-Prichard",1976-07-18,Chris Amon
3,133,1960-06-19,Lance Reventlow,1986-10-26,Alan Jones
4,85,1975-03-01,Guy Tunmer,1997-10-26,Gerhard Berger
5,77,1987-04-12,"Satoru Nakajima, Adrián Campos, Pascal Fabre",2012-11-25,Michael Schumacher
6,68,1997-06-15,Alexander Wurz,2022-07-31,"Lewis Hamilton, Fernando Alonso, Sebastian Vettel, Sergio Pérez, Daniel Ricciardo"
7,36,2013-03-17,"Max Chilton, Esteban Gutiérrez, Valtteri Bottas, Giedo van der Garde, Jules Bianchi",2022-07-31,"Valtteri Bottas, Kevin Magnussen, Max Verstappen, Carlos Sainz, Esteban Ocon, Lance Stroll, Pierre Gasly, Charles Leclerc, Lando Norris, George Russell, Alexander Albon, Nicholas Latifi, Yuki Tsunoda, Mick Schumacher, Guanyu Zhou"


In [32]:
farinaNumbers[["no_champs", "champions"]]

,no_champs,champions
0,1,Nino Farina
1,3,"Mike Hawthorn, Juan Fangio, Alberto Ascari"
2,5,"Graham Hill, Jim Clark, John Surtees, Phil Hill, Jack Brabham"
3,9,"Niki Lauda, Alan Jones, Mario Andretti, Jody Scheckter, James Hunt, Emerson Fittipaldi, Jackie Stewart, Jochen Rindt, Denny Hulme"
4,5,"Alain Prost, Ayrton Senna, Nigel Mansell, Nelson Piquet, Keke Rosberg"
5,4,"Michael Schumacher, Mika Häkkinen, Jacques Villeneuve, Damon Hill"
6,6,"Lewis Hamilton, Jenson Button, Sebastian Vettel, Kimi Räikkönen, Nico Rosberg, Fernando Alonso"
7,1,Max Verstappen
